In [1]:
# Okay time for our K-Means algorithm! woop woop!

# We want our Kmeans algorithm not only to cluster by k given cluster, it should also be able to find best k
# this will be done by an approach from the distortion theory, but later more on that. For now we just need to keep
# that in mind for our approach tactic

# 1. We need to take k+1 (starting with k=1) data points with k_max =< amount of values. (k_1+1,...,k_max) 

# 2. Calculate the euclidean distance to all data points and asign each to the class of the closest datapoint
# 3. Calculate each center of mass for this distribution
# 4. Repeat 2 and 3 m times until the centers of mass don't change and then continue with 1

# 5. Now just insert the results in the formula to best k by finding the lowest rate of distortion withing the calculated clusters

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from math import sqrt
from sklearn import datasets
from sklearn.datasets import make_blobs

# Create a clustered dataset to use
ds,y = make_blobs(n_samples=12, n_features=3, centers=4)

# Just to have it also as a dataframe, maybe we want to work with this
# But I'd like to try it as a version of n dimensional numpy arrays, because i used a lot of pandas in the previous exercises
df = pd.DataFrame (ds, y)
df['cluster'] = y
df

,0,1,2,cluster
1,-3.852681,-3.808310,-0.064839,1
3,1.746339,-1.905851,8.460040,3
0,1.664480,0.135146,-3.225799,0
2,1.611037,-4.666085,-0.546046,2
3,3.375221,0.066820,7.817567,3
0,4.359855,-0.308422,-3.191804,0
2,1.450221,-3.460404,-1.651012,2
2,1.118456,-3.737970,0.428257,2
3,1.706042,-2.570971,7.626164,3
1,-2.896500,-3.817314,0.582327,1


In [3]:
# 1. Get random starting points.
# when the logic on the simple model works, we will just embedd an incrementer on k and then compare the results for different k's
# only important thing till now is, k has to be changable and the code stil needs to work

clusters = []
centroids = {}
k = 3
i=0
l = 0

sample_numbers = random.sample(range(1, len(df)), k)

while i < k:
    centroids[i] = ds[sample_numbers[i]]
    clusters.append(list(centroids[i]))
    i = i + 1                 #else: call next function
    
# fine now we have a list of centroids in our list of clusters
clusters

[[1.4502208906125396, -3.460404108058833, -1.6510121644371998],
 [4.359855036162113, -0.30842225184530014, -3.191803855985228],
 [1.1184555392461966, -3.737970329018588, 0.42825725594428976]]

In [4]:
def euclidean_distance_on_array(target, distance_to):
    distance = 0.0
    for n in range(0,k-1): 
        distance = distance + (target[n] - distance_to[n])**2
    return sqrt(distance)

euclidean_distance_on_array(centroids[0], ds[3])


1.2163582609911485

In [5]:
def clustering(clusterspoints, dataset):   
    j = 0
    store = []  
    while j < k:     
        for clpo in clusterspoints:
            for row in dataset:
                store.append((row,euclidean_distance_on_array(row,clpo)))
        j = j+1
        return (store)

## Okay we got a list of arrays of the distances and datapoints to each cluster.

## Contining questions: 
###Work on the list, with k as an index for comparisment and then asign them to the center of clusters
###Or Seperate in three different lists by k values and then compare them
###assumption:
####the first approach would spare storage because this doesn't need to copy data and with indexing
####it's also going through the list fast with n/k simple comparisments of k values

#####!!!! With the first approach, storing the date in list_of_clusters needs to be specified by the amount
##### of the position with an if/else condition on: (len(list)/amount(clusters)) = 1, then cluster 0 ...

# Was heavier then expected with keeping it as dynamic as possible (for now)

distanced = clustering(clusters, ds)
distanced

[(array([-3.85268126, -3.80831019, -0.06483917]), 5.314302386957911),
 (array([ 1.74633919, -1.90585142,  8.46003996]), 1.5825043808532966),
 (array([ 1.66447966,  0.13514617, -3.22579936]), 3.6019284543406576),
 (array([ 1.61103699, -4.66608466, -0.54604614]), 1.2163582609911485),
 (array([3.37522127, 0.06681971, 7.81756734]), 4.018324818023943),
 (array([ 4.35985504, -0.30842225, -3.19180386]), 4.289634073303608),
 (array([ 1.45022089, -3.46040411, -1.65101216]), 0.0),
 (array([ 1.11845554, -3.73797033,  0.42825726]), 0.43256358536649),
 (array([ 1.70604229, -2.57097065,  7.62616357]), 0.9254925545918841),
 (array([-2.89649978, -3.81731352,  0.58232729]), 4.361348983099581),
 (array([ 2.11244088,  0.06275904, -2.95491542]), 3.584858976785329),
 (array([-1.69679251, -4.39023017, -1.15810003]), 3.281504207377522),
 (array([-3.85268126, -3.80831019, -0.06483917]), 8.927203817942242),
 (array([ 1.74633919, -1.90585142,  8.46003996]), 3.0630450579343576),
 (array([ 1.66447966,  0.13514617

In [6]:
#s = size of an average cluster. I had it calculated inplace before, but on large data it'll spare some calculations i guess
s = (len(distanced)/k)
int(s)
l = 0
dist_to_cluster=[]

stored = {}
while l < k:
    stored[l] = distanced[int(s*l)+int(s-1)]
    l = l+1

##Proof if the maths is correct
#while l < k:
#    print(int((len(distanced)/k)*l),int((len(distanced)/k)*l)+int((len(distanced)/k))-1)
#    print(l)
#    l = l+1


In [10]:
#Make a quick list of clusters so we can store the closest neighbor easily
list_of_clusters=[]
for row in clusters:
    list_of_clusters.append([row])
list_of_clusters

[[[1.4502208906125396, -3.460404108058833, -1.6510121644371998]],
 [[4.359855036162113, -0.30842225184530014, -3.191803855985228]],
 [[1.1184555392461966, -3.737970329018588, 0.42825725594428976]]]

In [11]:
#We have now the problem, I don't know and don't find an answer how to access the second item of the arrays
# in the dictonary so we will just create a pd.dataframe as a small help.
stored

{0: (array([-1.69679251, -4.39023017, -1.15810003]), 3.281504207377522),
 1: (array([-1.69679251, -4.39023017, -1.15810003]), 7.303706962143328),
 2: (array([-1.69679251, -4.39023017, -1.15810003]), 2.8898208409429835)}

In [13]:
pdf = pd.DataFrame(data=stored)
pdf

,0,1,2
0,"[-1.6967925133164554, -4.390230166269516, -1.1...","[-1.6967925133164554, -4.390230166269516, -1.1...","[-1.6967925133164554, -4.390230166269516, -1.1..."
1,3.2815,7.30371,2.88982
